## 라이브러리

In [3]:
import os
from dotenv import load_dotenv
from google import genai 
from google.genai import types
import json
import textwrap
import time
import base64
import dataclasses
from io import BytesIO
import numpy as np
from PIL import Image, ImageColor, ImageDraw, ImageFont
from typing import Tuple
import IPython
from IPython import display
import re

In [ ]:
#.env 파일 로드; 유효한 키 불러오기
load_dotenv("env/.env")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY_HW")

#사용예시
client = genai.Client(api_key=GOOGLE_API_KEY)

## Util 함수

In [9]:
def parse_json(json_output):
  # Parsing out the markdown fencing
  lines = json_output.splitlines()
  for i, line in enumerate(lines):
    if line == "```json":
      # Remove everything before "```json"
      json_output = "\n".join(lines[i + 1 :])
      # Remove everything after the closing "```"
      json_output = json_output.split("```")[0]
      break  # Exit the loop once "```json" is found
  return json_output


def get_image_resized(img_path):
    img = Image.open(img_path)
    img = img.resize(
        (800, int(800 * img.size[1] / img.size[0])), Image.Resampling.LANCZOS
    )
    return img


def generate_point_html(pil_image, points_json):
  buffered = BytesIO()
  pil_image.save(buffered, format="PNG")
  img_str = base64.b64encode(buffered.getvalue()).decode()
  points_json = parse_json(points_json)

  return f"""
<!DOCTYPE html>
<html>
<head>
    <title>Point Visualization</title>
    <style>
        body {{
            margin: 0;
            padding: 0;
            background: #fff;
            color: #000;
            font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif;
        }}

        .point-overlay {{
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            pointer-events: none;
        }}

        .point {{
            position: absolute;
            width: 12px;
            height: 12px;
            background-color: #2962FF;
            border: 2px solid #fff;
            border-radius: 50%;
            transform: translate(-50%, -50%);
            box-shadow: 0 0 40px rgba(41, 98, 255, 0.6);
            opacity: 0;
            transition: all 0.3s ease-in;
            pointer-events: auto;
        }}

        .point.visible {{
            opacity: 1;
        }}

        .point.fade-out {{
            animation: pointFadeOut 0.3s forwards;
        }}

        .point.highlight {{
            transform: translate(-50%, -50%) scale(1.1);
            background-color: #FF4081;
            box-shadow: 0 0 40px rgba(255, 64, 129, 0.6);
            z-index: 100;
        }}

        @keyframes pointFadeOut {{
            from {{
                opacity: 1;
            }}
            to {{
                opacity: 0.7;
            }}
        }}

        .point-label {{
            position: absolute;
            background-color: #2962FF;
            color: #fff;
            font-size: 14px;
            padding: 4px 12px;
            border-radius: 4px;
            transform: translate(20px, -10px);
            white-space: nowrap;
            opacity: 0;
            transition: all 0.3s ease-in;
            box-shadow: 0 0 30px rgba(41, 98, 255, 0.4);
            pointer-events: auto;
            cursor: pointer;
        }}

        .point-label.visible {{
            opacity: 1;
        }}

        .point-label.fade-out {{
            opacity: 0.45;
        }}

        .point-label.highlight {{
            background-color: #FF4081;
            box-shadow: 0 0 30px rgba(255, 64, 129, 0.4);
            transform: translate(20px, -10px) scale(1.1);
            z-index: 100;
        }}
    </style>
</head>
<body>
    <div id="container" style="position: relative;">
        <canvas id="canvas" style="background: #000;"></canvas>
        <div id="pointOverlay" class="point-overlay"></div>
    </div>

    <script>
        function annotatePoints(frame) {{
            // Add points with fade effect
            const pointsData = {points_json};

            const pointOverlay = document.getElementById("pointOverlay");
            pointOverlay.innerHTML = "";

            const points = [];
            const labels = [];

            pointsData.forEach(pointData => {{
                // Skip entries without coodinates.
                if (!(pointData.hasOwnProperty("point")))
                  return;

                const point = document.createElement("div");
                point.className = "point";
                const [y, x] = pointData.point;
                point.style.left = `${{x/1000.0 * 100.0}}%`;
                point.style.top = `${{y/1000.0 * 100.0}}%`;

                const pointLabel = document.createElement("div");
                pointLabel.className = "point-label";
                pointLabel.textContent = pointData.label;
                point.appendChild(pointLabel);

                pointOverlay.appendChild(point);
                points.push(point);
                labels.push(pointLabel);

                setTimeout(() => {{
                    point.classList.add("visible");
                    pointLabel.classList.add("visible");
                }}, 0);

                // Add hover effects
                const handleMouseEnter = () => {{
                    // Highlight current point and label
                    point.classList.add("highlight");
                    pointLabel.classList.add("highlight");

                    // Fade out other points and labels
                    points.forEach((p, idx) => {{
                        if (p !== point) {{
                            p.classList.add("fade-out");
                            labels[idx].classList.add("fade-out");
                        }}
                    }});
                }};

                const handleMouseLeave = () => {{
                    // Remove highlight from current point and label
                    point.classList.remove("highlight");
                    pointLabel.classList.remove("highlight");

                    // Restore other points and labels
                    points.forEach((p, idx) => {{
                        p.classList.remove("fade-out");
                        labels[idx].classList.remove("fade-out");
                    }});
                }};

                point.addEventListener("mouseenter", handleMouseEnter);
                point.addEventListener("mouseleave", handleMouseLeave);
                pointLabel.addEventListener("mouseenter", handleMouseEnter);
                pointLabel.addEventListener("mouseleave", handleMouseLeave);
            }});
        }}

        // Initialize canvas
        const canvas = document.getElementById("canvas");
        const ctx = canvas.getContext("2d");
        const container = document.getElementById("container");

        // Load and draw the image
        const img = new Image();
        img.onload = () => {{
            const aspectRatio = img.height / img.width;
            canvas.width = 800;
            canvas.height = Math.round(800 * aspectRatio);
            container.style.width = canvas.width + "px";
            container.style.height = canvas.height + "px";

            ctx.drawImage(img, 0, 0, canvas.width, canvas.height);

            frame.width = canvas.width;
            frame.height = canvas.height;
            annotatePoints(frame);
        }};
        img.src = "data:image/png;base64,{img_str}";

        const frame = {{
            width: canvas.width,
            height: canvas.height
        }};

        annotatePoints(frame);
    </script>
</body>
</html>
"""



additional_colors = [
    colorname for (colorname, colorcode) in ImageColor.colormap.items()
]


def plot_bounding_boxes(img, bounding_boxes):
  """Plots bounding boxes on an image.

  Plots bounding boxes on an image with markers for each a name, using PIL,
  normalized coordinates, and different colors.

  Args:
      img_path: The path to the image file.
      bounding_boxes: A list of bounding boxes containing the name of the object
        and their positions in normalized [y1 x1 y2 x2] format.
  """

  # Load the image
  width, height = img.size
  print(img.size)
  # Create a drawing object
  draw = ImageDraw.Draw(img)

  # Define a list of colors
  colors = [
      "red",
      "green",
      "blue",
      "yellow",
      "orange",
      "pink",
      "purple",
      "brown",
      "gray",
      "beige",
      "turquoise",
      "cyan",
      "magenta",
      "lime",
      "navy",
      "maroon",
      "teal",
      "olive",
      "coral",
      "lavender",
      "violet",
      "gold",
      "silver",
  ] + additional_colors

  # Parsing out the markdown fencing
  bounding_boxes = parse_json(bounding_boxes)

#   font = ImageFont.truetype("LiberationSans-Regular.ttf", size=14) #수정_260117
  try:
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=14)
  except IOError:
    font = ImageFont.load_default()

  # Iterate over the bounding boxes
  for i, bounding_box in enumerate(json.loads(bounding_boxes)):
    # Select a color from the list
    color = colors[i % len(colors)]

    # Convert normalized coordinates to absolute coordinates
    abs_y1 = int(bounding_box["box_2d"][0] / 1000 * height)
    abs_x1 = int(bounding_box["box_2d"][1] / 1000 * width)
    abs_y2 = int(bounding_box["box_2d"][2] / 1000 * height)
    abs_x2 = int(bounding_box["box_2d"][3] / 1000 * width)

    if abs_x1 > abs_x2:
      abs_x1, abs_x2 = abs_x2, abs_x1

    if abs_y1 > abs_y2:
      abs_y1, abs_y2 = abs_y2, abs_y1

    # Draw the bounding box
    draw.rectangle(((abs_x1, abs_y1), (abs_x2, abs_y2)), outline=color, width=4)

    # Draw the text
    if "label" in bounding_box:
      draw.text(
          (abs_x1 + 8, abs_y1 + 6), bounding_box["label"], fill=color, font=font
      )

  # Display the image
  img.show()


@dataclasses.dataclass(frozen=True)
class SegmentationMask:
  # bounding box pixel coordinates (not normalized)
  y0: int  # in [0..height - 1]
  x0: int  # in [0..width - 1]
  y1: int  # in [0..height - 1]
  x1: int  # in [0..width - 1]
  mask: np.array  # [img_height, img_width] with values 0..255
  label: str


def parse_segmentation_masks(
    predicted_str: str, *, img_height: int, img_width: int
) -> list[SegmentationMask]:
  items = json.loads(parse_json(predicted_str))
  masks = []
  for item in items:
    raw_box = item["box_2d"]
    abs_y0 = int(item["box_2d"][0] / 1000 * img_height)
    abs_x0 = int(item["box_2d"][1] / 1000 * img_width)
    abs_y1 = int(item["box_2d"][2] / 1000 * img_height)
    abs_x1 = int(item["box_2d"][3] / 1000 * img_width)
    if abs_y0 >= abs_y1 or abs_x0 >= abs_x1:
      print("Invalid bounding box", item["box_2d"])
      continue
    label = item["label"]
    png_str = item["mask"]
    if not png_str.startswith("data:image/png;base64,"):
      print("Invalid mask")
      continue
    png_str = png_str.removeprefix("data:image/png;base64,")
    png_str = base64.b64decode(png_str)
    mask = Image.open(BytesIO(png_str))
    bbox_height = abs_y1 - abs_y0
    bbox_width = abs_x1 - abs_x0
    if bbox_height < 1 or bbox_width < 1:
      print("Invalid bounding box")
      continue
    mask = mask.resize(
        (bbox_width, bbox_height), resample=Image.Resampling.BILINEAR
    )
    np_mask = np.zeros((img_height, img_width), dtype=np.uint8)
    np_mask[abs_y0:abs_y1, abs_x0:abs_x1] = mask
    masks.append(
        SegmentationMask(abs_y0, abs_x0, abs_y1, abs_x1, np_mask, label)
    )
  return masks


def overlay_mask_on_img(
    img: Image, mask: np.ndarray, color: str, alpha: float = 0.7
) -> Image.Image:
  """Overlays a single mask onto a PIL Image using a named color.

  The mask image defines the area to be colored. Non-zero pixels in the
  mask image are considered part of the area to overlay.

  Args:
      img: The base PIL Image object.
      mask: A PIL Image object representing the mask. Should have the same
        height and width as the img. Modes '1' (binary) or 'L' (grayscale) are
        typical, where non-zero pixels indicate the masked area.
      color: A standard color name string (e.g., 'red', 'blue', 'yellow').
      alpha: The alpha transparency level for the overlay (0.0 fully
        transparent, 1.0 fully opaque). Default is 0.7 (70%).

  Returns:
      A new PIL Image object (in RGBA mode) with the mask overlaid.

  Raises:
      ValueError: If color name is invalid, mask dimensions mismatch img
                  dimensions, or alpha is outside the 0.0-1.0 range.
  """
  if not (0.0 <= alpha <= 1.0):
    raise ValueError("Alpha must be between 0.0 and 1.0")

  # Convert the color name string to an RGB tuple
  try:
    color_rgb: Tuple[int, int, int] = ImageColor.getrgb(color)
  except ValueError as e:
    # Re-raise with a more informative message if color name is invalid
    raise ValueError(
        f"Invalid color name '{color}'. Supported names are typically HTML/CSS "
        f"color names. Error: {e}"
    )

  # Prepare the base image for alpha compositing
  img_rgba = img.convert("RGBA")
  width, height = img_rgba.size

  # Create the colored overlay layer
  # Calculate the RGBA tuple for the overlay color
  alpha_int = int(alpha * 255)
  overlay_color_rgba = color_rgb + (alpha_int,)

  # Create an RGBA layer (all zeros = transparent black)
  colored_mask_layer_np = np.zeros((height, width, 4), dtype=np.uint8)

  # Mask has values between 0 and 255, threshold at 127 to get binary mask.
  mask_np_logical = mask > 127

  # Apply the overlay color RGBA tuple where the mask is True
  colored_mask_layer_np[mask_np_logical] = overlay_color_rgba

  # Convert the NumPy layer back to a PIL Image
  colored_mask_layer_pil = Image.fromarray(colored_mask_layer_np, "RGBA")

  # Composite the colored mask layer onto the base image
  result_img = Image.alpha_composite(img_rgba, colored_mask_layer_pil)

  return result_img


def plot_segmentation_masks(
    img: Image, segmentation_masks: list[SegmentationMask]
):
  """Plots bounding boxes on an image.

  Plots bounding boxes on an image with markers for each a name, using PIL,
  normalized coordinates, and different colors.

  Args:
      img: The PIL.Image.
      segmentation_masks: A string encoding as JSON a list of segmentation masks
        containing the name of the object, their positions in normalized [y1 x1
        y2 x2] format, and the png encoded segmentation mask.
  """
  # Define a list of colors
  colors = [
      "red",
      "green",
      "blue",
      "yellow",
      "orange",
      "pink",
      "purple",
      "brown",
      "gray",
      "beige",
      "turquoise",
      "cyan",
      "magenta",
      "lime",
      "navy",
      "maroon",
      "teal",
      "olive",
      "coral",
      "lavender",
      "violet",
      "gold",
      "silver",
  ] + additional_colors

  font = ImageFont.load_default()

  # Do this in 3 passes to make sure the boxes and text are always visible.

  # Overlay the mask
  for i, mask in enumerate(segmentation_masks):
    color = colors[i % len(colors)]
    img = overlay_mask_on_img(img, mask.mask, color)

  # Create a drawing object
  draw = ImageDraw.Draw(img)

  # Draw the bounding boxes
  for i, mask in enumerate(segmentation_masks):
    color = colors[i % len(colors)]
    draw.rectangle(
        ((mask.x0, mask.y0), (mask.x1, mask.y1)), outline=color, width=4
    )

  # Draw the text labels
  for i, mask in enumerate(segmentation_masks):
    color = colors[i % len(colors)]
    if mask.label != "":
      draw.text((mask.x0 + 8, mask.y0 - 20), mask.label, fill=color, font=font)
  return img


def overlay_points_on_frames(original_frames, points_data_per_frame):
  """Overlays points on original frames and returns the modified frames."""
  modified_frames = []

  # Define colors for drawing points (using a consistent color per label for clarity)
  label_colors = {}
  current_color_index = 0
  available_colors = [
      "red",
      "green",
      "blue",
      "yellow",
      "orange",
      "pink",
      "purple",
      "brown",
      "gray",
      "beige",
      "turquoise",
      "cyan",
      "magenta",
      "lime",
      "navy",
      "maroon",
      "teal",
      "olive",
      "coral",
      "lavender",
      "violet",
      "gold",
      "silver",
  ]

  font = ImageFont.load_default()

  # Check if the number of original frames matches the number of processed data entries
  if len(original_frames) != len(points_data_per_frame):
    print(
        f"Error: Number of original frames ({len(original_frames)}) does not "
        "match the number of processed point data entries"
        f" ({len(points_data_per_frame)}). Cannot overlay points accurately."
    )
    return original_frames  # Return original frames if data doesn't match
  else:
    # Iterate through the frames and draw points
    for i, frame_pil in enumerate(original_frames):
      # Ensure frame is in RGB mode for drawing
      img = frame_pil.convert("RGB")
      draw = ImageDraw.Draw(img)
      width, height = img.size

      frame_points = points_data_per_frame[i]

      # Draw points on the frame
      for point_info in frame_points:
        if "point" in point_info and "label" in point_info:
          y_norm, x_norm = point_info["point"]
          label = point_info["label"]

          # Get color for the label
          if label not in label_colors:
            label_colors[label] = available_colors[
                current_color_index % len(available_colors)
            ]
            current_color_index += 1
          color = label_colors[label]

          # Convert normalized coordinates to absolute pixel coordinates
          abs_x = int(x_norm / 1000.0 * width)
          abs_y = int(y_norm / 1000.0 * height)

          # Draw a circle at the point
          point_radius = 5
          draw.ellipse(
              (
                  abs_x - point_radius,
                  abs_y - point_radius,
                  abs_x + point_radius,
                  abs_y + point_radius,
              ),
              fill=color,
              outline=color,
          )

          # Draw the label
          # Adjust label position to avoid going out of bounds
          label_pos_x = abs_x + point_radius + 2
          label_pos_y = (
              abs_y - point_radius - 10
              if abs_y - point_radius - 10 > 0
              else abs_y + point_radius + 2
          )
          draw.text((label_pos_x, label_pos_y), label, fill=color, font=font)

      # Append the modified PIL Image
      modified_frames.append(img)

    print(f"Processed and drew points on {len(modified_frames)} frames.")
    return modified_frames


def display_gif(frames_to_display):
  """Saves and displays a list of PIL Images as a GIF."""
  if frames_to_display:
    try:
      # Save the modified frames as a new GIF
      # output_gif_path = "/tmp/annotated_aloha_pen.gif" #수정_260117
      os.makedirs("output", exist_ok=True) # 폴더 없으면 만들기 (에러 방지)
      output_gif_path = "output/annotated_aloha_pen.gif"
      # Duration per frame in milliseconds (adjust as needed, 40ms is 25fps)
      duration_ms = 40
      # Ensure all frames are in RGB mode before saving as GIF
      rgb_frames = [frame.convert("RGB") for frame in frames_to_display]
      if rgb_frames:
        rgb_frames[0].save(
            output_gif_path,
            save_all=True,
            append_images=rgb_frames[1:],
            duration=duration_ms,
            loop=0,
        )

        # Display the GIF in Colab
        display.display(display.Image(output_gif_path))
        print(f"Displayed annotated GIF: {output_gif_path}")
      else:
        print("No frames to create GIF.")

    except Exception as e:
      print(f"Error creating or displaying annotated GIF: {e}")
  else:
    print("No frames to display.")


def extract_frames(gif):
  """Extracts frames from a GIF and returns a list of PIL Image objects."""
  frames = []
  try:
    while True:
      # Convert each frame to RGB to ensure compatibility with drawing
      frame = gif.convert("RGB")
      frames.append(frame)
      gif.seek(gif.tell() + 1)  # Move to the next frame
  except EOFError:
    pass  # End of sequence

  print(f"Extracted {len(frames)} frames from the GIF.")

  return frames


def populate_points_for_all_frames(total_frames, step, analyzed_data):
  """Populates point data for all frames based on analyzed frames."""
  points_data_all_frames = []
  analyzed_data_index = 0
  for i in range(total_frames):
    if i % step == 0 and analyzed_data_index < len(analyzed_data):
      points_data_all_frames.append(analyzed_data[analyzed_data_index])
      analyzed_data_index += 1
    else:
      # For frames that were not analyzed, use the data from the last analyzed
      # frame or append an empty list if no frame has been analyzed yet
      if analyzed_data_index > 0:
        points_data_all_frames.append(analyzed_data[analyzed_data_index - 1])
      else:
        # Should not happen if frames list is not empty
        points_data_all_frames.append([])
  return points_data_all_frames

In [11]:
# --- 1. 저장 함수 (헬퍼) ---
def save_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"✅ {filename} 저장 완료")


# --- 2. json변환 함수 --- 
def parse_gemini_json(response_obj):
    # 1. 응답 객체에서 텍스트 추출
    if hasattr(response_obj, 'text') and response_obj.text:
        raw_text = response_obj.text
    elif isinstance(response_obj, str) and response_obj:
        raw_text = response_obj # 이미 문자열인 경우 대비
    else:
        print("⚠️ 경고: AI 응답이 비어있거나 올바른 객체가 아닙니다.")
        return None

    try:
        # 2. 마크다운 클렌징
        clean_text = raw_text.replace("```json", "").replace("```", "").strip()
        
        # 3. 파싱 시도
        parsed_data = json.loads(clean_text)
        print("✅ JSON 파싱 성공!")
        return parsed_data
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: 형식이 올바르지 않습니다.\n{e}")
        # 디버깅을 위해 문제의 텍스트 앞부분만 출력
        print(f"문제가 된 텍스트(일부):\n{clean_text[:200]}...")
        return None

# 시작

## 워크플로우
1. 입력 (Raw Images): 5_camera.jpg, 6_camera.jpg ↓
2. 분석 (Perception):
    ◦ Gemini Robotics → [y, x] 좌표 획득
    ◦ Gemini 3 Pro → Z값, 회전각 획득 ↓
3. 병합 (Processing): Python 코드에서 두 정보를 합침 (vr JSON 생성) ↓
4. 전송 및 생성 (Action):
    ◦ VR 서버로 전송: 이 JSON을 그대로 쏘면 VR 엔진이 주사위를 배치합니다.
    ◦ GLB 생성: 이 JSON을 generate_glb.py에 넣으면 3D 모델 파일이 만들어집니다. 

## STEP1: [y, x] 좌표 획득
Robotics (Top): Red_Dice가 2개 감지됨 -> X 좌표가 작은 순서대로 Red_Dice_1, Red_Dice_2로 이름 변경.

In [55]:
#Step 1: Gemini Robotics (Top-View) 
# [y_norm, x_norm]: (세로, 가로), 0 (왼쪽/위쪽 끝) ~ 1000 (오른쪽/아래쪽 끝)

img_top = Image.open("./input/9_TopView.jpg")
prompt_top = textwrap.dedent("""
    **Task:** Detect all dice inside the blue tape area.
    
    **Labeling Rules:**
    - Assign a label based on the color of the dice.
    - Allowed Colors: ["Red", "Green", "Blue", "Yellow", "Orange", "White", "Pink"]
    - Strict Format: "{Color}_Dice" (e.g., "Red_Dice", "White_Dice").
    - You MUST output the list of objects sorted spatially from **LEFT to RIGHT** in the image.
    - List the **Bottom** object first, then the Top object.
    
    **Output Format:**
    Return a strictly valid JSON list of objects.
    - Each item must have a "point" and a "label".
    - "point": Must be in [y, x] format, normalized to 0-1000 scale.

    **Example Output:**
    [
      {"point": , "label": "Red_Dice"},
      {"point": , "label": "Red_Dice"},
      {"point": , "label": "White_Dice"}
    ]
    """)

start_time = time.time()

client_robo = genai.Client(api_key=GOOGLE_API_KEY)
json_top = client_robo.models.generate_content(
    model="gemini-robotics-er-1.5-preview",
    contents=[
        img_top,
        prompt_top,
    ],
    config=types.GenerateContentConfig(
        temperature=0.5,
        thinking_config=types.ThinkingConfig(thinking_budget=0),
    )
)
print(json_top.text)
print(f"\nTotal processing time: {(time.time() - start_time):.4f} seconds")

```json
[
  {"point": [453, 372], "label": "Pink_Dice"},
  {"point": [619, 372], "label": "Green_Dice"},
  {"point": [665, 396], "label": "Blue_Dice"},
  {"point": [619, 480], "label": "Orange_Dice"},
  {"point": [434, 483], "label": "Yellow_Dice"},
  {"point": [140, 383], "label": "Red_Dice"},
  {"point": [549, 640], "label": "White_Dice"}
]
```

Total processing time: 2.6088 seconds


In [56]:
robotics_data = []

# 파싱 로직
if json_top.text:
    try:
        raw_text = json_top.text
        clean_text = raw_text.replace("```json", "").replace("```", "").strip()
        
        robotics_data = json.loads(clean_text)
        print("✅ JSON 파싱 성공!")
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: 형식이 올바르지 않습니다.\n{e}")
        print(f"문제가 된 텍스트:\n{clean_text}")
else:
    print("⚠️ 경고: AI 응답이 비어있습니다. (Safety Filter에 걸렸을 수 있음)")

✅ JSON 파싱 성공!


In [57]:
if robotics_data:
    output_dir = "./output/"
    os.makedirs(output_dir, exist_ok=True)
    robotics_data_path = os.path.join(output_dir, "52_step1_robotics_raw.json")
    save_json(robotics_data, robotics_data_path) # 파일로 백업 (안전장치)

✅ ./output/52_step1_robotics_raw.json 저장 완료


## STEP2: Z값, 회전각 획득
Pro (Side): 이미지에서 왼쪽부터 오른쪽으로 주사위를 읽으라고 지시 -> 순서대로 Red_Dice_1, Red_Dice_2로 매핑.

In [58]:
#Step 2: Gemini 3 Pro (Side-View)
#일관성이 없으면: top에서 나온 데이터를 주입해서 활용하게 할 것

img_high = Image.open("./input/10_QuarterView.jpg")

prompt_high = textwrap.dedent("""\
    **Role:** You are a 3D Spatial Analysis Agent.
    
    **Task:** 
    Analyze the image (Side-View) to extract 3D attributes of ALL dice visible in the scene.
    
    **CRITICAL ORDERING RULE (MUST FOLLOW):**
    - You MUST output the list of objects sorted spatially from **LEFT to RIGHT** in the image.
    - If objects are stacked vertically, list the **Bottom** object first, then the Top object.
    
    **Extraction Requirements:**
    1. **Identify & Label:** 
       - Detect all dice regardless of color.
       - Determine the color of each die dynamically (e.g., Red, Blue, Green, Yellow, Orange, White, Pink).
       - **Label Format:** Strict adherence to "{Color}_Dice" (e.g., "Blue_Dice", "Yellow_Dice").
       
    2. **Z-Level:** Determine the height in millimeters. (Assume 1 dice height = 20mm).
       - On the table: `z_mm = 0`
       - On top of another dice: `z_mm = 20` (and so on).
       
    3. **Rotation:** Estimate the rotation (Yaw) in degrees relative to the camera frame.
    
    **Output Format:**
    Return a strictly valid JSON list.
    Example (Do not copy values, just the structure):
    [
      { "label": "Blue_Dice", "z_mm": 0, "rotation": 15 },
      { "label": "Yellow_Dice", "z_mm": 20, "rotation": 0 }
    ]
    
    """)
start_time = time.time()

client_pro = genai.Client(api_key=GOOGLE_API_KEY)
json_high = client_pro.models.generate_content(
    model="gemini-3-pro-preview",
    contents=[
        img_high,
        prompt_high,
    ],
    config=types.GenerateContentConfig(
        temperature=0.4,
        thinking_config=types.ThinkingConfig(thinking_budget=1024),
        # tools=[types.Tool(code_execution=types.ToolCodeExecution)], #제거 ->> Gemini가 코드를 짜고 자기가 알아서 실행 
        # response_mime_type="application/json" #선택 ->> JSON 포맷 강제
    )
)
print(json_high.text)
print(f"\nTotal processing time: {(time.time() - start_time):.4f} seconds")

Based on the visual analysis of the image, here are the 3D attributes for the dice, sorted from left to right.

```json
[
  {
    "label": "White_Dice",
    "z_mm": 0,
    "rotation": 45
  },
  {
    "label": "Orange_Dice",
    "z_mm": 0,
    "rotation": 0
  },
  {
    "label": "Yellow_Dice",
    "z_mm": 0,
    "rotation": 30
  },
  {
    "label": "Blue_Dice",
    "z_mm": 0,
    "rotation": 10
  },
  {
    "label": "Green_Dice",
    "z_mm": 0,
    "rotation": 0
  },
  {
    "label": "Pink_Dice",
    "z_mm": 0,
    "rotation": 45
  },
  {
    "label": "Red_Dice",
    "z_mm": 0,
    "rotation": 0
  }
]
```

Total processing time: 43.9590 seconds


In [59]:
pro_data = []

if json_high.text:
    try:
        match = re.search(r"```json(.*?)```", json_high.text, re.DOTALL)
        
        if match:
            clean_text = match.group(1).strip()
        else:
            clean_text = json_high.text.strip()
        
        pro_data = json.loads(clean_text)
        print("✅ JSON 파싱 성공!")
        print(pro_data)
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: 형식이 올바르지 않습니다.\n{e}")
        # print(f"❌ JSON 파싱 실패::\n{clean_text}") # 필요시 주석 해제
else:
    print("⚠️ 경고: AI 응답이 비어있습니다.")

✅ JSON 파싱 성공!
[{'label': 'White_Dice', 'z_mm': 0, 'rotation': 45}, {'label': 'Orange_Dice', 'z_mm': 0, 'rotation': 0}, {'label': 'Yellow_Dice', 'z_mm': 0, 'rotation': 30}, {'label': 'Blue_Dice', 'z_mm': 0, 'rotation': 10}, {'label': 'Green_Dice', 'z_mm': 0, 'rotation': 0}, {'label': 'Pink_Dice', 'z_mm': 0, 'rotation': 45}, {'label': 'Red_Dice', 'z_mm': 0, 'rotation': 0}]


In [60]:
if pro_data:
    output_dir = "./output/"
    os.makedirs(output_dir, exist_ok=True)
    pro_data_path = os.path.join(output_dir, "52_step2_pro_attributes.json")
    save_json(pro_data, pro_data_path) # 파일로 백업 (안전장치)

✅ ./output/52_step2_pro_attributes.json 저장 완료


## STEP3: label 기준 1차 병합
stack_i(Unique ID) 구분: **"Top-View의 X,Y 좌표가 겹치면(Threshold) 같은 ID를 부여하고, 높이(Level) 순서는 Side-View를 본 Gemini에게 물어보는 것"**입니다. 이것이 물리적 좌표의 정확성와 시각적 추론의 유연성을 모두 잡는 방법입니다.

In [61]:
#Step 3: Python 병합 (Merging)

# 파이썬 병합 코드 (1:N 매칭 지원)
# 이 코드는 Top-View의 좌표를 Side-View의 층수(Level)만큼 복제(Duplicate)하여 병합합니다.

def merge_stacked_dice(robotics_data, pro_data):
    merged_list = []
    
    # 1. Pro 데이터를 색상별로 그룹화 (Stacking 감지)
    pro_groups = {}
    for item in pro_data:
        label = item['label']
        if label not in pro_groups:
            pro_groups[label] = []
        pro_groups[label].append(item)

    # 2. Robotics(좌표) 데이터를 순회하며 매칭
    # 좌표 정렬: 왼쪽(X) -> 오른쪽
    robotics_sorted = sorted(robotics_data, key=lambda k: k['point'][1]) # [y, x]이므로 index 1이 x

    # 색상별로 Robotics에서 몇 번째 주사위인지 카운트
    robotics_counters = {} 

    for robo_item in robotics_sorted:
        color_label = robo_item['label']
        point = robo_item['point'] # 
        
        # 현재 색상의 주사위가 Pro 데이터에 있는지 확인
        if color_label in pro_groups:
            target_list = pro_groups[color_label]
            num_pro = len(target_list)
            pass 

    # 1. Robotics 데이터를 색상별 큐(Queue)로 만듦
    robo_queues = {}
    for item in sorted(robotics_data, key=lambda k: k['point'][1]):
        lbl = item['label']
        if lbl not in robo_queues: robo_queues[lbl] = []
        robo_queues[lbl].append(item['point'])
        
    # 2. Pro 데이터를 순회하며 좌표 할당
    final_objects = []
    
    last_label = None
    last_coord = None
    label_counters = {} #추가_260107
    
    for pro_item in pro_data:
        label = pro_item['label']
        z_mm = pro_item.get('z_mm', 0)
        
        # 좌표 가져오기 로직
        current_coord = None
        
        if label in robo_queues and robo_queues[label]:
            # 스택 감지: 같은 색상이고 Z가 0보다 크면(즉 2층이면), 방금 썼던 좌표 재사용
            if z_mm > 0 and last_label == label and last_coord is not None:
                 current_coord = last_coord
            else:
                # 1층이거나 새로운 주사위면 큐에서 하나 꺼냄 (없으면 1층 좌표 재사용하거나 예외처리)
                if robo_queues[label]:
                    current_coord = robo_queues[label] # 일단 봄
                    # Red(1층) -> 좌표 꺼냄(Pop)
                    # Red(2층) -> 좌표 재사용(Pop 안함)
                    
                    if z_mm == 0: # 새로운 바닥 주사위
                        current_coord = robo_queues[label].pop(0) 
                    else: # 쌓인 주사위 (이전 바닥 주사위 좌표 공유)
                        current_coord = last_coord 

        if current_coord:
            last_coord = current_coord
            last_label = label

            #추가_260107
            if 'unique_id' not in pro_item:
                count = label_counters.get(label, 0)
                unique_id = f"{label}_{count}"
                label_counters[label] = count + 1
            else:
                unique_id = pro_item['unique_id']
            
            # 병합된 객체 생성
            new_obj = pro_item.copy()
            new_obj['unique_id'] = unique_id #추가_260107
            new_obj['point'] = current_coord # [y, x]
            final_objects.append(new_obj)
            
    return final_objects

In [62]:
output_dir = "./output/"
os.makedirs(output_dir, exist_ok=True)

merged_data = merge_stacked_dice(robotics_data, pro_data) # 변수 사용
save_path = os.path.join(output_dir, "52_step3_merged_final.json")
save_json(merged_data, save_path)

✅ ./output/52_step3_merged_final.json 저장 완료


## (선택)STEP4: 이미지로 검증
▪ 쌓여 있는 주사위 위에 점이 찍혔는데, 라벨에 Z:1(2층)이라고 적혀 있는가?

▪ 살짝 돌아간 주사위 라벨에 Rot:45 등 0이 아닌 값이 적혀 있는가?

In [63]:
# [Step 4] 시각화 검증 (Visualization)

def overlay_points_on_frames(original_frames, points_data_per_frame):
    modified_frames = []
    available_colors = ["red", "green", "blue", "yellow", "orange", "white", "pink"]
    label_colors = {}
    current_color_index = 0
    font = ImageFont.load_default()

    for i, frame_pil in enumerate(original_frames):
        img = frame_pil.convert("RGB")
        draw = ImageDraw.Draw(img)
        width, height = img.size
        print(f"이미지 크기: {width}x{height}") # 디버깅용

        frame_points = points_data_per_frame[i]

        for point_info in frame_points:
            if "point" in point_info and "label" in point_info:
                y_norm, x_norm = point_info["point"]
                label_text = point_info["label"] # 수정된 라벨(속성 포함)

                # 색상 지정
                base_label = label_text.split('\n')[0] # 색상은 원래 이름(Red_Dice) 기준  #수정; 리스트에서 **첫 번째 요소(진짜 라벨 이름)**만 꺼내서 키로 써야 합니다.
                if base_label not in label_colors:
                    label_colors[base_label] = available_colors[current_color_index % len(available_colors)]
                    current_color_index += 1
                color = label_colors[base_label]

                # [Source 124] 좌표 변환 (0-1000 -> 픽셀)
                abs_x = int(x_norm / 1000.0 * width)
                abs_y = int(y_norm / 1000.0 * height)

                # 점 찍기
                r = 5
                draw.ellipse((abs_x - r, abs_y - r, abs_x + r, abs_y + r), fill=color, outline=color)

                # 라벨 그리기 (위치 조정)
                draw.text((abs_x + 8, abs_y - 10), label_text, fill=color, font=font)
        
        modified_frames.append(img)
    return modified_frames


In [64]:

# 병합된 최종 JSON 파일 로드
with open("./output/52_step3_merged_final.json", "r", encoding="utf-8") as f:
    merged_data = json.load(f)

# Top-View 이미지 로드 [Source 1]
image_path = "./input/9_TopView.jpg"
if not os.path.exists(image_path):
    print(f"⚠️ 경고: {image_path} 파일이 없습니다.")
    # 테스트용 빈 이미지 생성
    img_top = Image.new('RGB', (1000, 1000), color='black')
else:
    img_top = Image.open(image_path)

In [65]:

# --- 검증용 데이터 포맷팅 (Visual Formatting) ---
verification_data = []

for item in merged_data:
    # 3D 속성 정보를 텍스트 라벨에 욱여넣습니다.
    debug_label = (
        f"{item['label']}\n"       # 예: Red_Dice_0
        f"Z: {item['z_mm']}mm\n"       # 예: Z: 20mm (2층 확인용)
        f"Rot: {item['rotation']}°"    # 예: Rot: 45°
    )
    
    # 데이터 구조 맞추기
    verification_data.append({
        "point": item['point'], # [y, x] (Source 47 포맷)
        "label": debug_label,             # 화면에 출력될 텍스트
        # "label": f"{item['unique_id']}\nZ:{item['z_mm']}" #디버깅용_추후수정
    })

In [66]:
# --- 시각화 실행 및 저장 ---
annotated_frames = overlay_points_on_frames([img_top], [verification_data])

# 결과 확인
result_img = annotated_frames[0]
# result_img.show()  # 화면에 띄우기
result_img.save("./output/52_step4_verify_merge.jpg")
print("✅ 검증 이미지가 생성되었습니다.")

이미지 크기: 1920x1080
✅ 검증 이미지가 생성되었습니다.


## (선택)STEP4-2: AI로 역검증
• 1, 2단계에서 만든 **'점과 글자가 찍힌 검증용 이미지'**를 다시 Gemini에게 보냅니다.

In [ ]:
#ai에게 검증시키기 X
# Prompt: "Here is an image annotated with AI analysis results. Please audit this image. If the label says 'Z:20' (meaning 2nd floor), distinctively verify if the object is physically stacked on top of another. Report any discrepancies between the text label and the visual reality."

## STEP5: VR용 최종 병합

In [38]:
'''
최적의 VR 전송용 JSON 구조 (Recommended)

VR 엔진이 추가 연산 없이 바로 객체(Actor)를 생성할 수 있도록 World Transform(위치, 회전, 스케일) 중심으로 구성된 구조입니다.
{
  "timestamp": 1716345600,  // 동기화 시점 (선택)
  "scene_id": "table_top_01",
  "objects": [
    {
      "id": "Red_Dice_0",         // 고유 식별자 (필수)
      "type": "dice",             // 3D 모델 에셋 이름 (Prefabs/Dice)
      "properties": {
        "color": "red",           // 재질(Material) 변경용
        "stack_level": 0          // 로직 처리용 (선택)
      },
      "transform": {
        "position": { "x": 250.5, "y": 10.0, "z": 400.2 }, // mm 단위 월드 좌표
        "rotation": { "x": 0, "y": 45.0, "z": 0 },         // Euler Angles (Yaw 적용)
        "scale": { "x": 1, "y": 1, "z": 1 }                // 크기 변환 필요시
      }
    },
    {
      "id": "Red_Dice_1",
      "type": "dice",
      "properties": { "color": "red", "stack_level": 1 },
      "transform": {
        "position": { "x": 250.5, "y": 30.0, "z": 400.2 }, // 2층 (y + 20mm)
        "rotation": { "x": 0, "y": 45.0, "z": 0 },
        "scale": { "x": 1, "y": 1, "z": 1 }
      }
    }
  ]
}
'''

'\n최적의 VR 전송용 JSON 구조 (Recommended)\n\nVR 엔진이 추가 연산 없이 바로 객체(Actor)를 생성할 수 있도록 World Transform(위치, 회전, 스케일) 중심으로 구성된 구조입니다.\n{\n  "timestamp": 1716345600,  // 동기화 시점 (선택)\n  "scene_id": "table_top_01",\n  "objects": [\n    {\n      "id": "Red_Dice_0",         // 고유 식별자 (필수)\n      "type": "dice",             // 3D 모델 에셋 이름 (Prefabs/Dice)\n      "properties": {\n        "color": "red",           // 재질(Material) 변경용\n        "stack_level": 0          // 로직 처리용 (선택)\n      },\n      "transform": {\n        "position": { "x": 250.5, "y": 10.0, "z": 400.2 }, // mm 단위 월드 좌표\n        "rotation": { "x": 0, "y": 45.0, "z": 0 },         // Euler Angles (Yaw 적용)\n        "scale": { "x": 1, "y": 1, "z": 1 }                // 크기 변환 필요시\n      }\n    },\n    {\n      "id": "Red_Dice_1",\n      "type": "dice",\n      "properties": { "color": "red", "stack_level": 1 },\n      "transform": {\n        "position": { "x": 250.5, "y": 30.0, "z": 400.2 }, // 2층 (y + 20mm)\n        "rotation": { "x": 0, "y":

In [115]:
# [Step 5] VR 전송용 JSON
# Step 3에서 병합된 데이터를 위 형식으로 변환하는 함수입니다.

def format_for_vr(merged_data):
    vr_payload = {
        "timestamp": time.time(),
        "objects": []
    }
    
    # 설정값 (테이블 실제 크기: 예 150mm x 150mm)
    TABLE_WIDTH_MM = 570
    TABLE_DEPTH_MM = 570
    DICE_HEIGHT_MM = 20 # 주사위 중심 높이 (반지름 10mm 가정)

    for item in merged_data:
        # [y, x] 0-1000 정규화 좌표 -> mm 변환
        y_norm, x_norm = item['point']
        
        # Unity/Unreal 좌표계 (Y-up 기준)
        # X: 가로, Y: 높이, Z: 깊이
        world_x = (x_norm / 1000.0) * TABLE_WIDTH_MM
        world_z = (y_norm / 1000.0) * TABLE_DEPTH_MM
        
        # 높이 계산 (바닥에서 10mm 띄움 + 층수 * 20mm)
        # z_mm가 0(바닥)이면 중심은 10mm 높이
        world_y = (DICE_HEIGHT_MM / 2) + item['z_mm'] 

        vr_obj = {
            "id": item.get('unique_id', f"{item['label']}_{int(time.time())}"),
            "type": "dice",
            "properties": {
                "color": item['label'].split('_')[0].lower() # "Red_Dice" -> "red"
            },
            "transform": {
                "position": { "x": round(world_x, 1), "y": world_y, "z": round(world_z, 1) },
                "rotation": { "x": 0, "y": item['rotation'], "z": 0 },
                "scale": { "x": 1, "y": 1, "z": 1 }, #이미 20mm 크기의 모델 원본 크기 그대로(100%) 사용
            }
        }
        vr_payload["objects"].append(vr_obj)
        
    return vr_payload

In [117]:
json_merged = "./output/52_step3_merged_final.json"
with open(json_merged, "r", encoding="utf-8") as f:
    merged_data_source = json.load(f)
vr_result = format_for_vr(merged_data_source) 

output_dir = "./output/"
os.makedirs(output_dir, exist_ok=True)
save_path = os.path.join(output_dir, "52_step5_vr.json")
save_json(vr_result, save_path)

print(f"✅ VR 데이터 저장 완료")

✅ ./output/52_step5_vr.json 저장 완료
✅ VR 데이터 저장 완료


# 위 JSON결과물; **VR서버 전달**

In [124]:
# 검증
def verify_3d_logic(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
        
    objects = data.get('objects', [])
    print(f"🧐 총 {len(objects)}개의 객체 검증 시작...\n")
    
    # 1. 높이(Y) 일관성 체크
    y_values = [obj['transform']['position']['y'] for obj in objects]
    if len(set(y_values)) > 1:
        print(f"⚠️ 경고: 주사위 높이(Y)가 서로 다릅니다! {set(y_values)}")
    else:
        print(f"✅ 높이(Y) 검증 통과: 모두 {y_values[0]} 입니다.")

    # 2. 겹침(Collision) 체크
    # 주사위 크기가 20mm이므로, 중심 거리가 20mm보다 가까우면 겹친 것
    min_dist = 20.0 
    
    for i in range(len(objects)):
        for j in range(i + 1, len(objects)):
            p1 = objects[i]['transform']['position']
            p2 = objects[j]['transform']['position']
            
            # 유클리드 거리 계산 (X, Z 평면)
            dist = ((p1['x'] - p2['x'])**2 + (p1['z'] - p2['z'])**2)**0.5
            
            if dist < min_dist:
                print(f"⚠️ 충돌 경고: {objects[i]['id']} 와 {objects[j]['id']} 거리가 너무 가깝습니다 ({dist:.1f}mm)")

    print("\n검증 종료.")

# 실행
verify_3d_logic('./output/scene.json')

🧐 총 1개의 객체 검증 시작...

✅ 높이(Y) 검증 통과: 모두 10.0 입니다.

검증 종료.


## (건너뛰기)STEP6 -> STEP7 생성코드; 처음 한번만 실행

In [31]:
# [Step 6] GLB 생성(2)

# trimesh 또는 numpy 라이브러리
json_vr = "./output/51_step5_vr.json"
with open(json_vr, "r", encoding="utf-8") as f:
    json_vr = json.load(f) 
result_glb = "./output/51_dice_digital_twin.glb"

prompt_glb_reusable = textwrap.dedent(f"""\
    **Role:** Python 3D Developer
    **Task:** Write a Python script to generate '{result_glb}'.

    **Crucial Requirement (Dynamic Loading):**
    1. Do NOT hardcode the data inside the script.
    2. The script must **load** data from a generic JSON file named **'{json_vr}'**.
    3. Iterate through `data['objects']` and create 3D cubes using `trimesh`.
    - Map 'transform' > 'position' directly to the mesh.
    - Map 'properties' > 'color' to the mesh visual.

    **Output:** Provide the robust Python code.
    """)

start_time = time.time()

client_pro = genai.Client(api_key=GOOGLE_API_KEY)
json_glb_reusable = client_pro.models.generate_content(
    model="gemini-3-pro-preview",
    contents=[
        prompt_glb_reusable, 
    ],
    config=types.GenerateContentConfig(
        temperature=1.0, #코딩 능력 극대화
        thinking_config=types.ThinkingConfig(thinking_budget=1024), #좌표계 변환 로직 수
    )
)
print(json_glb_reusable.text) # 디버깅용 출력
print(f"\nTotal processing time: {(time.time() - start_time):.4f} seconds")


Here is the robust Python script.

To ensure the "Dynamic Loading" requirement is met while making the script runnable immediately, **I have added a setup step at the beginning of the script that creates the `data.json` file** containing the specific JSON string you provided. The script then reads from that file, ensuring the logic remains decoupled from the data.

### Prerequisites
You will need to install `trimesh` and `numpy`.
```bash
pip install trimesh numpy
```

### Python Script

```python
import trimesh
import json
import numpy as np
import os
import math

# ==========================================
# SETUP: SIMULATE EXTERNAL DATA FILE
# ==========================================
# In a real production environment, this file would already exist.
# We create it here to strictly satisfy the requirement of loading from a file
# without hardcoding the data into the logic loop.
INPUT_FILENAME = 'dice_data_input.json'
RAW_JSON_DATA = {
    'timestamp': 1770620809.666629,
    'object

In [33]:
output_dir = "./output/"
output_filename = "51_step6_make_glb.py"
full_path = os.path.join(output_dir, output_filename)

# 마크다운 태그 제거
clean_code = json_glb_reusable.text.replace("```python", "").replace("```", "").strip()
# 파일 쓰기
with open(full_path, "w", encoding="utf-8") as f:
    f.write(clean_code)

print(f"✅ Python script saved")

✅ Python script saved


## (실행)STEP7 -> glb 생성(주사위 전용 3D)
step6에서 만든 파일 수정: 하나의 파일로 다양한 json 데이터 기반 glb 파일 생성 위함

In [ ]:
# 3d 생성 Python Script

# pip install trimesh numpy scipy

import trimesh
import numpy as np
import json
import os

# Configuration
INPUT_FILENAME = './output/52_step5_vr.json'
OUTPUT_PATH = './output/52_step7_digital_twin.glb'

def get_color_rgba(color_name):
    cmap = {
        'red':   [255, 0, 0, 255],
        'white': [240, 240, 240, 255],
        'blue':  [0, 0, 255, 255],
        'green': [0, 255, 0, 255],
        "yellow": [255, 255, 0, 255],
        "orange": [255, 165, 0, 255],
        "pink":   [255, 192, 203, 255],
        # 'black': [20, 20, 20, 255],
    }
    return cmap.get(color_name.lower(), [128, 128, 128, 255]) # Default gray

def process_dice_twin(json_input_path, glb_output_path):
    # 1. Dynamic Loading: Read from external JSON file
    if not os.path.exists(json_input_path):
        raise FileNotFoundError(f"Input file not found: {json_input_path}")
        
    with open(json_input_path, 'r') as f:
        data = json.load(f)

    # Initialize Scene
    scene = trimesh.Scene()
    
    # 2. Iterate through objects
    objects = data.get('objects', [])
    print(f"Processing {len(objects)} objects...")

    for obj in objects:
        obj_id = obj.get('id', 'unknown')
        props = obj.get('properties', {})
        trans = obj.get('transform', {})
        
        # Geometry Creation
        # 주사위 크기 조절_2600209
        # mesh = trimesh.creation.box(extents=[0.02, 0.02, 0.02]) # 예: 높이가 4.0이면, 위로 2.0, 아래로 2.0 뻗어 있음

        #수정_2600209; glb에 정보 json과 같이 넣기
        box_size = 0.02 
        mesh = trimesh.creation.box(extents=[box_size, box_size, box_size])
        mesh.apply_translation([0, box_size / 2.0, 0]) #상자를 높이의 절반만큼 위로 올려

        # Apply Color
        color_name = props.get('color', 'gray')
        rgba = get_color_rgba(color_name)
        mesh.visual.face_colors = rgba
        
        # -------------------------------------------------
        # Transform Logic
        # -------------------------------------------------        
        # Position
        pos_data = trans.get('position', {'x': 0, 'y': 0, 'z': 0})
        # translation = [pos_data['x'], pos_data['y'], pos_data['z']] #_2600209 크기 cm로 바꿀거라
        scale_factor = 0.001
        translation = [
            pos_data['x'] * scale_factor, 
            pos_data['y'] * scale_factor, 
            pos_data['z'] * scale_factor
        ]

        # #=========3 -> 높이 변경 X
        # target_y = pos_data['y'] * scale_factor
        # HALF_HEIGHT = box_size / 2.0 # 10mm (중심 보정용)
        # if target_y < HALF_HEIGHT:
        #     extra_h = item.get('z_mm', 0) * 0.001 
        #     target_y = HALF_HEIGHT + extra_h
        # #=========
        
        # Rotation (Euler Degrees -> Radians -> Matrix)
        rot_data = trans.get('rotation', {'x': 0, 'y': 0, 'z': 0})
        # Converting degrees to radians
        euler_angles = np.radians([rot_data['x'], rot_data['y'], rot_data['z']])
        # Create rotation matrix (Assuming standard XYZ order)
        matrix_rot = trimesh.transformations.euler_matrix(
            euler_angles[0], euler_angles[1], euler_angles[2], axes='sxyz'
        )
        
        # Scale
        scale_data = trans.get('scale', {'x': 1, 'y': 1, 'z': 1})
        matrix_scale = np.eye(4)
        matrix_scale[0,0] = scale_data['x']
        matrix_scale[1,1] = scale_data['y']
        matrix_scale[2,2] = scale_data['z']
        
        # Translation Matrix
        matrix_trans = trimesh.transformations.translation_matrix(translation)
        
        # Combine Matrices: Translation * Rotation * Scale
        # Note: Trimesh handles multiplication order for us usually, 
        # but logically it is T @ R @ S
        final_transform = trimesh.transformations.concatenate_matrices(
            matrix_trans, matrix_rot, matrix_scale
        )
        
        # Apply transform to mesh
        # mesh.apply_transform(final_transform)
        
        # Add to scene
        # scene.add_geometry(mesh, node_name=obj_id)
        scene.add_geometry(mesh, node_name=obj_id, transform=final_transform)

    # 3. Export
    # Ensure directory exists
    output_dir = os.path.dirname(glb_output_path)
    if output_dir and not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir, exist_ok=True)
            print(f"Created directory: {output_dir}")
        except OSError as e:
            print(f"Error creating directory: {e}")
            return

    print(f"Exporting GLB to: {glb_output_path}")
    scene.export(glb_output_path)
    print("Generation Complete.")

if __name__ == "__main__":
    # Run the generator
    if os.path.exists(INPUT_FILENAME):
        process_dice_twin(INPUT_FILENAME, OUTPUT_PATH)
        print(f"🎉 GLB 생성 완료! 저장 경로: {OUTPUT_PATH}")
    else:
        print(f"⚠️ 오류: 데이터 파일이 없습니다. {INPUT_FILENAME}")

Processing 7 objects...
Exporting GLB to: ./output/52_step7_digital_twin.glb
Generation Complete.
🎉 GLB 생성 완료! 저장 경로: ./output/52_step7_digital_twin.glb


In [122]:
#검증
import trimesh
import numpy as np

# 1. 확인하고 싶은 파일 로드
mesh = trimesh.load('./output/scene.glb', force='mesh')

# Scene인 경우 내부의 첫 번째 물체를 꺼냅니다.
if isinstance(mesh, trimesh.Scene):
    # Scene 내의 모든 형상(Geometry)을 합쳐서 전체 크기를 봅니다.
    mesh = mesh.dump(concatenate=True) 

# 2. 크기 출력 (mm 단위라고 가정)
size = mesh.extents  # [x길이, y길이, z길이] 반환
print(f"📏 모델 크기 (Dimensions):")
print(f" - X (가로): {size[0]:.2f} units") 
print(f" - Y (높이): {size[1]:.2f} units") #M
print(f" - Z (세로): {size[2]:.2f} units")
print(f" - 전체 크기: {size}") 

# 3. 20mm 검증
if np.allclose(size, [3], atol=1.0):
    print("✅ 확인: 이 모델은 약 20mm 크기입니다.")
else:
    print("⚠️ 주의: 크기가 20mm가 아닙니다! 스케일 조정이 필요할 수 있습니다.")

📏 모델 크기 (Dimensions):
 - X (가로): 0.02 units
 - Y (높이): 0.02 units
 - Z (세로): 0.02 units
 - 전체 크기: [0.02 0.02 0.02]
⚠️ 주의: 크기가 20mm가 아닙니다! 스케일 조정이 필요할 수 있습니다.


In [123]:
#검증
import trimesh
import numpy as np

# 1. 파일 로드
scene = trimesh.load('./output/scene.glb', force='scene')

# 2. 첫 번째 주사위만 꺼내기
# scene.geometry는 딕셔너리 형태입니다 (예: {'geometry_0': <Trimesh>})
keys = list(scene.geometry.keys())
if len(keys) > 0:
    first_geometry_key = keys[0] 
    first_dice = scene.geometry[first_geometry_key]
    
    # 3. 그 녀석의 크기 재기
    size = first_dice.extents # [x, y, z]

    print(f"🎲 개별 주사위 크기: {size}")

    # 검증
    if np.allclose(size, [0.02, 0.02, 0.02], atol=0.001):
        print("✅ 완벽한 20mm 정육면체입니다.")
    else:
        print(f"⚠️ 크기 확인 필요: {size}")

else:
    print("⚠️ 씬(Scene)에 형상이 하나도 없습니다.")


🎲 개별 주사위 크기: [0.02 0.02 0.02]
✅ 완벽한 20mm 정육면체입니다.


# 위 GLB결과물; **VR서버 전달** 

In [ ]:
# 로봇팔사진으로 검증